# Why MongoDB?

MongoDB is a document based NoSQL that stores the documents in a JSON-like format:

  * single CSV, collection of documents (records for each athlete)
  * size of the collections is not a problem (horizontal scale out)
  * no need to normalize (remove needs for joins) exploiting the JSON format
  * having all stored in a single collection makes easier (and faster) to retrieve info
  * thought for DBs with no need to be frequently updated (it's still possible but cannot update multiple docs)
    

# Initial Dataset

Idea:

Instead of denormalizing as in SQL, keep a single collection were the nested attributes are metadata referring to the same topic. Replacing SQL tables exploiting nested structure of JSON documents. In practice with this powerful tool we've been able to avoid normalization by replacing the 'tables' to keep them inside a single collection.

_for example:_  __attribute athlete contains anagraphic informations such as Name, Sex and Age__

In [ ]:
import pandas as pd
df = pd.read_csv(r'/home/arturo/Desktop/DM/HW2/csv/results.csv', low_memory = False)

print(df.shape)
df.head()

(1412370, 29)


,_id,Name,Sex,Event,Division,Date,MeetCountry,MeetName,Equipment,Age,...,Deadlift1Kg,Deadlift2Kg,Deadlift3Kg,Deadlift4Kg,TotalKg,Place,Wilks,McCulloch,Glossbrenner,IPFPoints
0,AbbieMurphyFSBDFOR20181027AustraliaMelbourneCup,Abbie Murphy,F,SBD,F-OR,2018-10-27,Australia,Melbourne Cup,Wraps,29.0,...,110.0,120.0,130.0,0.0,290.0,4,324.16,324.16,286.42,511.15
1,AbbieTuongFSBDFOR20181027AustraliaMelbourneCup,Abbie Tuong,F,SBD,F-OR,2018-10-27,Australia,Melbourne Cup,Wraps,29.0,...,130.0,140.0,145.0,0.0,332.5,2,378.07,378.07,334.16,595.65
2,AinsleeHooperFBFOR20181027AustraliaMelbourneCup,Ainslee Hooper,F,B,F-OR,2018-10-27,Australia,Melbourne Cup,Raw,40.0,...,0.0,0.0,0.0,0.0,32.5,1,38.56,38.56,34.12,313.97
3,AmyMoldenhauerFSBDFOR20181027AustraliaMelbourn...,Amy Moldenhauer,F,SBD,F-OR,2018-10-27,Australia,Melbourne Cup,Wraps,23.0,...,132.5,0.0,0.0,0.0,310.0,3,345.61,345.61,305.37,547.04
4,AndreaRowanFSBDFOR20181027AustraliaMelbourneCup,Andrea Rowan,F,SBD,F-OR,2018-10-27,Australia,Melbourne Cup,Wraps,45.0,...,150.0,160.0,170.0,0.0,390.0,3,321.25,338.91,274.56,550.08


# Setting up

In [ ]:
import pymongo
import pprint
import time
from bson.son import SON

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client.powerlifting

placement = db.placement
meet = db.meet
mapping = db.mapping
records = db.records

# Queries

# 1. Athlete with most first places (MySQL: 0.8 sec)

select a.athlname, count(id) as wins

from athlete as a join results as r on a.id = r.athlete

where place = '1'

group by athlname

order by count(id) desc

limit 1;

In [ ]:
# AGGREGATION PIPELINE

start = time.time()
out = records.aggregate([
    { '$match': { 'results.Place': { '$eq': '1' } } }, 
    { '$group': { '_id': '$athlete.Name', 'Wins': { '$sum': 1 } } }, 
    { '$sort': { 'Wins': -1 } } ] )

for i in list(out)[:1]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60)

{'_id': 'Sverre Paulsen', 'Wins': 161}

The time of execution of above query is : 0.1406938115755717


In [ ]:
# WITH LOOKUP

start = time.time()
out = placement.aggregate([
        {"$match": {"Place": {"$eq" : 1}}},
        {'$lookup':
            {'from': "mapping",
            'localField': "_id",
            'foreignField': "_id",
            'as': "infos"}},
        {'$unwind': '$infos'},
        {'$group': {"_id": '$infos.Name', "Wins": {"$sum": 1}}},
        {"$sort": {"Wins": -1}}])

for i in list(out)[:1]: print(i)
end = time.time()

print()
print("The time of execution of above query is:", (end-start)/60)

{'_id': 'Sverre Paulsen', 'Wins': 161}

The time of execution of above query is: 1.049993872642517


In [ ]:
# WITH MAP-REDUCE (deprecated in MongoDb 5.0, suggested to use aggregation pipelines)

db.records.mapReduce(
    function() {emit(this.athlete.Name, 1)},
    function(key, values) {return Array.sum(values)},
    {
        query: {"results.Place": {"$eq": '1'}},
        out: 'wins'
    }
)

db.wins.find().sort({value: -1}).limit(1)

# 2. Average results of competitions he took part in (MySQL: 0.02 sec)
select r.athlete, round(avg(r.age), 2) avg_age, round(avg(r.bodyweight), 2) avg_weight, round(avg(r.totalkg), 2) avg_totalkg, round(avg(r.place), 2) avg_place

from results as r join athlete as a on a.id = r.athlete 

where a.athlname = 'Sverre Paulsen'

group by r.athlete;


In [ ]:
# AGGREGATION PIPELINE

start = time.time()
out = records.aggregate([
    { '$match': { '$and': [ { 'athlete.Name': { '$eq': 'Sverre Paulsen' } }, { 'results.Place': { '$ne': 'DQ' } } ] } }, 
    { '$group': { '_id': '$athlete.Name', 
                    'Avg age': { '$avg': '$athlete.Age' }, 
                    'Avg weight': { '$avg': '$athlete.BodyweightKg' }, 
                    'Avg totalkg': { '$avg': '$results.TotalKg' }, 
                    'Avg place': { '$avg': { '$toInt': '$results.Place' } } } } ] )

for i in list(out): print(i)
end = time.time()

print()
print("The time of execution of above query is:", (end-start)/60)

{'_id': 'Sverre Paulsen', 'Avg age': 51.43882978723404, 'Avg weight': 70.39117021276596, 'Avg totalkg': 203.41755319148936, 'Avg place': 1.175531914893617}

The time of execution of above query is: 0.013660518328348796


In [ ]:
# WITH LOOKUP

start = time.time()
out = placement.aggregate([
        {'$lookup':
            {'from': "mapping",
             'localField': "_id",
             'foreignField': "_id",
             'as': "infos"}},
        {'$unwind': {'path': '$infos'}},
        {'$match':{'infos.Name': 'Sverre Paulsen'}},
        {"$group": {"_id": "$infos.Name",
                    "Avg age": {"$avg": "$Age"}, 
                    "Avg weight": {"$avg": "$BodyweightKg"}, 
                    "Avg totalkg": {"$avg": "$TotalKg"}, 
                    "Avg place": {"$avg": "$Place"}}}])

for i in list(out)[:1]: print(i)
end = time.time()

print()
print("The time of execution of above query is:", (end-start)/60)

{'_id': 'Sverre Paulsen', 'Avg age': 51.46596858638743, 'Avg weight': 70.37193717277488, 'Avg totalkg': 200.22251308900525, 'Avg place': 1.175531914893617}

The time of execution of above query is: 2.8395498633384704


# 3. Countries with most meetings (MySQL: 0.02 sec)

select meetcountry, count(meetcountry) as meets

from meet

group by meetcountry

order by count(meetcountry) desc

limit 20;

In [ ]:
# AGGREGATION PIPELINE

start = time.time()
out = meet.aggregate([
        {"$group": {"_id": "$MeetCountry", "Number of meets": {"$sum": 1}}},
        {"$sort": SON([("Number of meets", -1), ("_id", -1)])}])

for i in list(out)[:5]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60)

{'_id': 'USA', 'Number of meets': 9690}
{'_id': 'Norway', 'Number of meets': 2378}
{'_id': 'Australia', 'Number of meets': 1291}
{'_id': 'Canada', 'Number of meets': 1233}
{'_id': 'Finland', 'Number of meets': 882}

The time of execution of above query is : 0.002012165387471517


In [ ]:
# WITH MAP-REDUCE (deprecated in MongoDb 5.0, suggested to use aggregation pipelines)

db.meet.mapReduce(
    function() {emit(this.MeetCountry, 1)},
    function(key, values) {return Array.sum(values)},
    {
        out: 'totmeets'
    }
)

db.totmeets.find().sort({value: -1}).limit(10)

# 4. Meet where the youngest athletes competed (MySQL: 0.02)
select m.*

from results as r join meet as m on m.id = r.meet

where r.age = (select min(age)
			   from ageclass)

order by meetdate desc;

In [ ]:
# AGGREGATION PIPELINE

start = time.time()
out = list(records.aggregate([
    { '$match': {'athlete.Age': { '$eq': 5 } } },
    { '$project': { '_id': 0, 'MeetName': '$meet.MeetName', 'MeetCountry': '$meet.MeetCountry', 'Date': '$meet.Date'} } ], allowDiskUse=True ) )
    
for i in out[:5]: print(i)

end = time.time()
print()
print("The time of execution of above query is :", (end-start)/60)  

{'MeetName': 'Iron Disorder Near the Border', 'MeetCountry': 'USA', 'Date': '2019-02-23'}
{'MeetName': 'World Championships', 'MeetCountry': 'Canada', 'Date': '1996-03-16'}
{'MeetName': 'Mississippi State', 'MeetCountry': 'USA', 'Date': '1993-04-06'}
{'MeetName': 'So Cal Meet', 'MeetCountry': 'USA', 'Date': '2017-02-25'}
{'MeetName': 'Australian Schools Postal Bench Press', 'MeetCountry': 'Australia', 'Date': '2002-01-01'}

The time of execution of above query is : 0.01971426010131836


In [ ]:
# WITH LOOKUP

start = time.time()
out = placement.aggregate([
        {"$sort": SON([("Age", 1), ("_id", 1)])},
        {"$match" : {"Age" : {"$gt":  0.0}}},
        {'$lookup' :
            {'from': "mapping",
             'localField': "_id",
             'foreignField': "_id",
             'as': "infos"}},
        {'$unwind': {'path': '$infos'}},
        {'$project': {"_id": 0, "infos.MeetName": 1, "infos.MeetCountry": 1, "infos.Date": 1}}], 
        allowDiskUse=True)

for i in list(out)[:1]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60)

{'infos': {'Date': '2006-07-22', 'MeetCountry': 'USA', 'MeetName': 'Single Lift Nationals'}}

The time of execution of above query is : 1.684627385934194


# 5. Name and weight of athletes who weight more than the average, order by bodyweight (MySQL: 0.32 sec)
select distinct a.athlname, round(avg(r.bodyweight), 2)

from results as r join athlete as a on a.id = r.athlete

where r.bodyweight > (select avg(r1.bodyweight)
					   from results as r1)

group by a.athlname

order by avg(r.bodyweight) desc;


In [ ]:
# AGGREGATION PIPELINE

start = time.time()
average = list(records.aggregate([
                {"$match": {"athlete.BodyweightKg": {"$gt": 0}}},
                {"$group": {"_id": "null", "avg weight": { "$avg": "$athlete.BodyweightKg"}}}]))[0]['avg weight']
print("The average weight is:", average, '\n')

out = list(records.aggregate([
                {"$match" : {"athlete.BodyweightKg": {"$gt": average}}}, 
                {"$project": {'athlete.Name' : 1, 'athlete.BodyweightKg': 1, '_id': 0 }}, 
                {"$group": {"_id": "$athlete.Name",  "BodyweightKg" : { "$first": '$athlete.BodyweightKg'}}}, 
                {"$sort" : {"BodyweightKg" : -1}}], allowDiskUse=True))

for i in out[:10]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60)  

The average weight is: 84.19788743333208 

{'_id': 'Jousha Sprouse', 'BodyweightKg': 258.0}
{'_id': 'James Simmons', 'BodyweightKg': 242.3}
{'_id': 'Jeff Lewis', 'BodyweightKg': 240.4}
{'_id': 'Joshua Mcelhaney', 'BodyweightKg': 232.7}
{'_id': 'Daniel Defonseca', 'BodyweightKg': 230.0}
{'_id': 'Nathan Baptist', 'BodyweightKg': 227.5}
{'_id': 'Isaac Doss', 'BodyweightKg': 227.5}
{'_id': 'Christopher Sutton', 'BodyweightKg': 227.25}
{'_id': 'Anthony Callier', 'BodyweightKg': 226.8}
{'_id': 'Scott Solon', 'BodyweightKg': 222.3}

The time of execution of above query is : 0.12059585253397624


# 6. Names of athletes over 20 years old who compete in the "Open" or "Boys" category (MySQL: 1.5 sec)
select a.athlname, avg(r.age)

from results as r join athlete as a on a.id = r.athlete

where r.age > 20 and (r.division = "Open" or r.division= "Boys")

group by a.athlname

limit 100;



In [ ]:
# AGGREGATION PIPELINE

start = time.time()
out = records.distinct("athlete.Name", {"$and": 
                [{"$or": [{"competition.Division": "Open"}, {"competition.Division": "Boys"}]}, 
                 {"athlete.Age": {"$gt": 20}}]})

for i in list(out)[:5]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60) 


A Abduzhabarov
A DziewiczKiewicz
A Eginov
A Esrekov
A Jay Montanez

The time of execution of above query is : 0.03082797924677531


In [ ]:
# WITH LOOKUP

start = time.time()
out = mapping.aggregate([
        {'$match': {'$or':[{"Division": "Open"}, {"Division": "Boys"}]}},
        {'$lookup':
            {'from': "placement",
             'localField': "_id",
             'foreignField': "_id",
             'as': "infos"}},
        {'$unwind': {'path': '$infos'}},
        {'$match': {'infos.Age': {"$gt": 20}}},
        {"$sort": {"Name": 1}},
        {'$project': {"_id": 0, "Name": 1}}], allowDiskUse=True)

for i in list(out)[:5]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60)

{'Name': 'A Abduzhabarov'}
{'Name': 'A DziewiczKiewicz'}
{'Name': 'A Eginov'}
{'Name': 'A Esrekov'}
{'Name': 'A Jay Montanez'}

The time of execution of above query is : 1.4924104690551758


# 7. First attempts in squat, deadlift and bench for male athletes with "Raw" equipment (MySQL: 0.36 sec)
select a.athlname, round(avg(b.bench1), 2) avg_bench, round(avg(d.deadlift1),2) avg_deadlift, round(avg(s.squat1),2) avg_squat

from athlete as a join bench as b on a.id = b.athlete join deadlift as d on a.id = d.athlete and b.meet = d.meet join squat as s on a.id = s.athlete and b.meet = s.meet

where b.category = 'SBD' and d.equipment = "Raw" and a.sex = 'M'

group by athlname;

In [ ]:
# AGGREGATION PIPELINE

start = time.time()
out = list(records.aggregate([
    { '$match': { '$and': [ { 'athlete.Sex': 'M' }, { 'results.Equipment': 'Raw' }, { 'competition.Event': 'SBD' } ] } }, 
    { '$project': {
            '_id': 0, 
            'Name': '$athlete.Name', 
            'Squat': '$squat.Squat1Kg', 
            'Bench': '$bench.Bench1Kg', 
            'Deadlift': '$deadlift.Deadlift1Kg' } } ], allowDiskUse=True))

for i in out[:3]: print(i)       
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60) 

{'Name': 'Adrian Zwaan', 'Squat': 80.0, 'Bench': 0.0, 'Deadlift': 130.0}
{'Name': 'Ben Hales', 'Squat': 165.0, 'Bench': 107.5, 'Deadlift': 190.0}
{'Name': 'Brady Curtin', 'Squat': 160.0, 'Bench': 107.5, 'Deadlift': 200.0}

The time of execution of above query is : 0.027448185284932456


In [ ]:
# WITH LOOKUP

start = time.time()
out = mapping.aggregate([
        {'$match':{'Sex': 'M','Event': 'SBD'}},
        {'$lookup':
            {'from': "placement",
             'localField': "_id",
             'foreignField': "_id",
             'as': "infos"}},
        {'$unwind': {'path': '$infos'}},
        {'$match':{'infos.Equipment': 'Raw'}},
        {'$lookup':
            {'from': "squat",
             'localField': "_id",
             'foreignField': "_id",
             'as': "infos1"}},
        {'$unwind': {'path': '$infos1'}},
        {'$lookup' :
            {'from': "bench",
             'localField': "_id",
             'foreignField': "_id",
             'as': "infos2"}},
        {'$unwind': {'path': '$infos2'}},
        {'$lookup':
            {'from': "deadlift",
             'localField': "_id",
             'foreignField': "_id",
             'as': "infos3"}},
        {'$unwind': {'path': '$infos3'}},
        {'$project': {"_id": 0, "Name": 1, 'infos1.Squat1Kg':1, 'infos2.Bench1Kg':1, 'infos3.Deadlift1Kg': 1}}],
        allowDiskUse=True)

for i in list(out)[:5]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60)

{'Name': 'Adrian Zwaan', 'infos1': {'Squat1Kg': 80.0}, 'infos2': {'Bench1Kg': 0.0}, 'infos3': {'Deadlift1Kg': 130.0}}
{'Name': 'Ben Hales', 'infos1': {'Squat1Kg': 165.0}, 'infos2': {'Bench1Kg': 107.5}, 'infos3': {'Deadlift1Kg': 190.0}}
{'Name': 'Brady Curtin', 'infos1': {'Squat1Kg': 160.0}, 'infos2': {'Bench1Kg': 107.5}, 'infos3': {'Deadlift1Kg': 200.0}}
{'Name': 'Griffin Spence', 'infos1': {'Squat1Kg': 165.0}, 'infos2': {'Bench1Kg': 100.0}, 'infos3': {'Deadlift1Kg': 220.0}}
{'Name': 'Jim Collins', 'infos1': {'Squat1Kg': 90.0}, 'infos2': {'Bench1Kg': 67.5}, 'infos3': {'Deadlift1Kg': 155.0}}

The time of execution of above query is : 2.6795273303985594


# 8. Top 10 women who have higher wilk than average men wilk (MySQL: 3.3 sec)
select a.athlname, round(avg(s.wilks), 2)

from athlete a, scores s

where s.athlete = a.id and a.sex = 'F' and s.wilks > (select avg(s1.wilks)
                                                      from scores s1, athlete a1
													  where s1.athlete = a1.id and a1.sex = 'M')

group by a.athlname

order by round(avg(s.wilks), 2) desc

limit 10;

In [ ]:
# AGGREGATION PIPELINE

start = time.time()
avg_wilk_men = list(records.aggregate([
                    {"$match": {"$and": [{"athlete.Sex" : 'M'}, {"scores.Wilks": {"$gt": 0}}]}},
                    {"$group": {"_id": "null", "avg wilk man": {"$avg": "$scores.Wilks"}}}]))[0]['avg wilk man']

print("The average weight is:", avg_wilk_men, '\n')

out = list(records.aggregate([
    { '$match': { '$and': [ { 'athlete.Sex': { '$eq': 'F' } }, { 'scores.Wilks': { '$gt': avg_wilk_men } } ] } }, 
    { '$group': { '_id': '$athlete.Name', 'Average Wilk': { '$avg': '$scores.Wilks' } } }, 
    { '$sort': { 'Average Wilk': -1 } }, 
    { '$project': { '_id': 0, 'Name': '$_id', 'Average Wilk': 1 } } ] ) )

for i in out[:3]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60) 

The average weight is: 292.47849212315595 

{'Average Wilk': 690.52, 'Name': 'Becca Swanson'}
{'Average Wilk': 689.0505263157894, 'Name': 'Laura Phelps-Sweatt'}
{'Average Wilk': 636.58, 'Name': 'Widari Widari'}

The time of execution of above query is : 0.05147123336791992


# 9. meet with no athletes with less than 18 years (MySQL: 0.7 sec)
select m.id , m.meetname

from meet as m join results as r on m.id = r.meet

where r.meet not in (select r1.meet 
					 from results as r1 
					 where r1.age < 18 and r1.age > 0
					 group by r1.athlete) 

group by m.id;


In [ ]:
# AGGREGATION PIPELINE

start = time.time()
out = records.distinct('meet.MeetName', {"athlete.Age" : {"$gt" :  18 }})

for i in list(out[:10]): print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60)

015 Pennsylvania State Bench Press and Deadlift
1. Pitomača Open u powerliftingu & benchpressu
1. Runde Bayernliga
1. Runde Bayernliga Bankdrücken
1. Runde Bayernliga KDK
1. Runde Bundesliga Kraftdreikampf
1. Runde Sachsen-Ober-Liga im Bankdrücken
1. Wettkampftag Bundesliga Kraftdreikampf
10TH ANNUAL NAVASOTA INVATATIONAL
10th Annual Asa Chambless Powerlifting Classic

The time of execution of above query is : 0.016390486558278402


# 10. Athletes TotalKg above average but McCulloch score below average (MySQL: 3.7 sec)

select a.*

from athlete as a

WHERE EXISTS (select *
			  from results as r
			  where a.id = r.athlete and r.totalkg > (select avg(r1.totalkg)
													  from results as r1))

AND NOT EXISTS (select *
				from scores as s
				where a.id = s.athlete and s.wilks < (select avg(s1.wilks)
												      from scores as s1));

In [ ]:
# AGGREGATION PIPELINE

start = time.time()
avg_totkg = list(records.aggregate([
                {"$match": {"results.TotalKg": {"$gt": 0}}},
                {"$group": {"_id": "null", "avg totkg": {"$avg": '$results.TotalKg'}}}]))[0]['avg totkg']

print("The average totkg is:", avg_totkg, '\n')

avg_mc = list(records.aggregate([
               {"$match": {"scores.McCulloch": {"$gt": 0}}},
               {"$group": {"_id": "null", "avg mc": {"$avg": '$scores.McCulloch'}}}]))[0]['avg mc']

print("The average mc is:", avg_mc, '\n')

out = records.aggregate([{'$match': { "$and": [ { 'results.TotalKg': {"$gt": avg_totkg}  }, { 'scores.McCulloch': { "$lt": avg_mc } } ] } },
                        { '$group': { '_id': '$athlete.Name', 'Average TotKg': { '$avg': '$results.TotalKg' }, 'Average McCulloch': { '$avg': '$scores.McCulloch' } } }, 
                        { '$sort': { 'Average TotKg': -1 } }, 
                        { '$project': { '_id': 0, 'Name': '$_id', 'Average TotKg': 1 } } ] )   

for i in list(out)[:10]: print(i)
end = time.time()

print()
print("The time of execution of above query is :", (end-start)/60)  

The average totkg is: 396.1769361021555 

The average mc is: 296.4856997197737 

{'Average TotKg': 1191.01, 'Name': 'Henry Thomason'}
{'Average TotKg': 1178.2175, 'Name': 'Donnie Thompson #1'}
{'Average TotKg': 1165.0, 'Name': 'Kari Markku Kalliola'}
{'Average TotKg': 1155.3400000000001, 'Name': 'Scott Cartwright'}
{'Average TotKg': 1127.5, 'Name': 'Mike Ruggeria'}
{'Average TotKg': 1110.54, 'Name': 'Tyler Butcher'}
{'Average TotKg': 1105.63, 'Name': 'Mike Ruggiera'}
{'Average TotKg': 1100.0, 'Name': 'Bill Kazmaier'}
{'Average TotKg': 1097.5, 'Name': 'Chad Aichs'}
{'Average TotKg': 1090.0, 'Name': 'John Bernor'}

The time of execution of above query is : 0.10386797587076822
